In [ ]:
!pip install wandb


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports

%cd /content/drive/MyDrive/video_classification_rnn

import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
from imutils import paths
import pandas as pd
import os 

from load_video import load_video
from models import Models
import wandb
from wandb.keras import WandbCallback

/content/drive/MyDrive/video_classification_rnn


In [ ]:
# Download ucf101_top5 dataset

# !pip install -q git+https://github.com/tensorflow/docs
# !wget -q https://git.io/JGc31 -O /content/drive/MyDrive/dataset/ucf101_top5.tar.gz

In [4]:
# Parameters setting for ucf101_top5 dataset


ucf101_top5_data_path = ""
frame_height = frame_width = 112
BATCH_SIZE = 70
batch_size = 10
EPOCHS = 10
epochs = 15

max_seq_len = 20
NUM_FEATURES = 2048

In [7]:
# Create directories

if not os.path.exists("weights"):
    os.makedirs("weights")

## Prepairing data for ucf101_top5 dataset

In [5]:
%cd /content/drive/MyDrive/dataset/ucf101_top5


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

/content/drive/MyDrive/dataset/ucf101_top5
Total videos for training: 594
Total videos for testing: 224


,video_name,tag
493,v_TennisSwing_g10_c04.avi,TennisSwing
377,v_ShavingBeard_g10_c05.avi,ShavingBeard
43,v_CricketShot_g14_c02.avi,CricketShot
42,v_CricketShot_g14_c01.avi,CricketShot
580,v_TennisSwing_g23_c05.avi,TennisSwing
453,v_ShavingBeard_g22_c04.avi,ShavingBeard
59,v_CricketShot_g16_c04.avi,CricketShot
123,v_PlayingCello_g08_c06.avi,PlayingCello
45,v_CricketShot_g14_c04.avi,CricketShot
534,v_TennisSwing_g16_c03.avi,TennisSwing


In [6]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_ucf101_video(path, max_frames=0, resize=(frame_height, frame_width)):
    cap = cv2.VideoCapture(path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # if num_frames < max_seq_len:
    #   print(num_frames)

    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return frames, num_frames

In [7]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['CricketShot', 'PlayingCello', 'Punch', 'ShavingBeard', 'TennisSwing']


In [8]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)

    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, max_seq_len), dtype="bool")

    frames_dataset = []
    video_count = 0

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames, num_frames = load_ucf101_video(os.path.join(root_dir, path))
        # frames = frames[None, ...]

        padded_frames = np.zeros(shape=(max_seq_len, frame_height, frame_width, 3), dtype="float32")
        current_video_seq_len = min(max_seq_len, num_frames)

        # Normalize video frames
        for i, frame in enumerate(np.array(frames)):
            if i < max_seq_len:
               padded_frames[i, :] = frame / 255.

        frames_dataset.append(padded_frames)
        frame_masks[video_count, :current_video_seq_len] = 1
        video_count+=1

  # Convert to np.array
    frames_dataset = np.array(frames_dataset)
    labels = np.array(labels)


    return [frames_dataset, frame_masks], labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")


In [9]:
# Splite data

X_train, X_val, mask_train, mask_val, Y_train, Y_val = train_test_split(train_data[0], train_data[1], train_labels, test_size = 0.2, random_state=42)

print("train data shape: ", X_train.shape, Y_train.shape, mask_train.shape, "validation data shape: ",  X_val.shape, mask_val.shape, Y_val.shape)

train data shape:  (475, 20, 112, 112, 3) (475, 1) (475, 20) validation data shape:  (119, 20, 112, 112, 3) (119, 20) (119, 1)


### Define Models, Compile and fit

01- RNN

In [ ]:
wandb.init(project='Video Classification-CRNN')
#

In [11]:
config = wandb.config
config.learning_rate = 0.001

In [12]:
models = Models((frame_height, frame_width), max_seq_len)
rnn_model = models.RNN_model()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

rnn_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=batch_size, epochs=epochs, callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
48/48 [==============================] - 32s 350ms/step - loss: 1.3855 - accuracy: 0.5368 - val_loss: 2.2509 - val_accuracy: 0.4286 - _timestamp: 1652855046.0000 - _runtime: 38.0000
Epoch 2/15
48/48 [==============================] - 15s 307ms/step - loss: 0.6384 - accuracy: 0.8063 - val_loss: 1.6789 - val_accuracy: 0.4370 - _timestamp: 1652855060.0000 - _runtime: 52.0000
Epoch 3/15
48/48 [==============================] - 14s 298ms/step - loss: 0.3898 - accuracy: 0.8989 - val_loss: 1.7150 - val_accuracy: 0.4874 - _timestamp: 1652855075.0000 - _runtime: 67.0000
Epoch 4/15
48/48 [==============================] - 15s 313ms/step - loss: 0.4197 - accuracy: 0.9032 - val_loss: 0.8512 - val_accuracy: 0.7311 - _timestamp: 1652855089.0000 - _runtime: 81.0000
Epoch 5/15
48/48 [==============================] - 15s 312ms/step - loss: 0.2926 - accuracy: 0.9284 - val_loss: 0.6474 - val_accuracy: 0.7983 - _timestamp: 1652855105.0000 - _runtime: 97.0000
Epoch 6/15
48/48 [=================

#### 02- GRU

In [14]:
models = Models((frame_height, frame_width), max_seq_len)
rnn_model = models.RNN_model()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

rnn_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=batch_size, epochs=epochs, callbacks=[WandbCallback()])

Epoch 1/15
48/48 [==============================] - 19s 328ms/step - loss: 1.3624 - accuracy: 0.5516 - val_loss: 1.6691 - val_accuracy: 0.3361 - _timestamp: 1652855417.0000 - _runtime: 409.0000
Epoch 2/15
48/48 [==============================] - 15s 308ms/step - loss: 0.5531 - accuracy: 0.8674 - val_loss: 1.9209 - val_accuracy: 0.3025 - _timestamp: 1652855431.0000 - _runtime: 423.0000
Epoch 3/15
48/48 [==============================] - 15s 308ms/step - loss: 0.2964 - accuracy: 0.9432 - val_loss: 1.7538 - val_accuracy: 0.3782 - _timestamp: 1652855446.0000 - _runtime: 438.0000
Epoch 4/15
48/48 [==============================] - 15s 309ms/step - loss: 0.1991 - accuracy: 0.9663 - val_loss: 0.8959 - val_accuracy: 0.7143 - _timestamp: 1652855461.0000 - _runtime: 453.0000
Epoch 5/15
48/48 [==============================] - 15s 310ms/step - loss: 0.2284 - accuracy: 0.9600 - val_loss: 0.7717 - val_accuracy: 0.7311 - _timestamp: 1652855476.0000 - _runtime: 468.0000
Epoch 6/15
48/48 [============